In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.write_config(path='.azureml')
experiment_name='udacity-project'

exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

print(ws)

Workspace name: quick-starts-ws-158927
Azure region: southcentralus
Subscription id: 5a4ab2ba-6c51-4805-8155-58759ad589d8
Resource group: aml-quickstarts-158927
Workspace.create(name='quick-starts-ws-158927', subscription_id='5a4ab2ba-6c51-4805-8155-58759ad589d8', resource_group='aml-quickstarts-158927')


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "my-compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Cluster already exists. Use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)

Cluster already exists. Use it.

Running


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

In [4]:
from azureml.core import Environment, ScriptRunConfig

env = Environment.get(workspace = ws, name = "AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")

In [5]:
if "training" not in os.listdir():
    os.mkdir("./training")

In [6]:
# SKLearn estimator is deprecated. Use ScriptRunConfig instead.
"""
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./",
              compute_target=compute_target,
              vm_size="Standard_D2_V2",
              entry_script="train.py")
"""

'\n# Create a SKLearn estimator for use with train.py\nest = SKLearn(source_directory="./",\n              compute_target=compute_target,\n              vm_size="Standard_D2_V2",\n              entry_script="train.py")\n'

In [7]:
src = ScriptRunConfig(source_directory = ".",
                      script = "train.py",
                      compute_target = "my-compute-cluster",
                      environment = env
                     )

In [8]:
# Specify parameter sampler
ps = RandomParameterSampling({"--C":choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
                             "--max_iter":choice(50,100,200,300)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=1)

In [9]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hd_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             policy=policy,
                             max_total_runs=16,
                             max_concurrent_runs=4)

In [10]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hd_run = exp.submit(hd_config)

# The widget RunDetails can be used to monitor the run
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [11]:
hd_run.wait_for_completion(show_output=True)

RunId: HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b
Web View: https://ml.azure.com/runs/HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-158927/workspaces/quick-starts-ws-158927&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-09-21T09:59:54.385473][API][INFO]Experiment created<END>\n""<START>[2021-09-21T09:59:54.997678][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-09-21T09:59:55.187725][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b
Web View: https://ml.azure.com/runs/HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-158927/workspaces/quick-starts-ws-158927&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b',
 'target': 'my-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-21T09:59:54.080936Z',
 'endTimeUtc': '2021-09-21T10:11:38.83082Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '5e065b8c-6ca0-423b-b1a7-c0e1a2a1e085',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.33.0',
  'score': '0.9091047040971169',
  'best_child_run_id': 'HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b_1',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg158927.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b/azureml-logs/hyperdrive.txt?sv=2019-07-07&

In [12]:
import joblib
# Get your best run and save the model from that run.

print(hd_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

[{'run_id': 'HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b_1', 'hyperparameters': '{"--C": 50, "--max_iter": 300}', 'best_primary_metric': 0.9091047040971169, 'status': 'Completed'}, {'run_id': 'HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b_8', 'hyperparameters': '{"--C": 10, "--max_iter": 100}', 'best_primary_metric': 0.908649468892261, 'status': 'Completed'}, {'run_id': 'HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b_5', 'hyperparameters': '{"--C": 1000, "--max_iter": 100}', 'best_primary_metric': 0.908649468892261, 'status': 'Completed'}, {'run_id': 'HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b_12', 'hyperparameters': '{"--C": 500, "--max_iter": 100}', 'best_primary_metric': 0.9084977238239758, 'status': 'Completed'}, {'run_id': 'HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b_9', 'hyperparameters': '{"--C": 50, "--max_iter": 100}', 'best_primary_metric': 0.9084977238239758, 'status': 'Completed'}, {'run_id': 'HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b_11', 'hyperparameters': '{"--C": 0.001, "--max_iter": 100}', 'bes

In [25]:
# Best model
print(hd_run.get_children_sorted_by_primary_metric(top=1, reverse=False, discard_no_metric=False))

[{'run_id': 'HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b_1', 'hyperparameters': '{"--C": 50, "--max_iter": 300}', 'best_primary_metric': 0.9091047040971169, 'status': 'Completed'}]


In [13]:
best_run = hd_run.get_best_run_by_primary_metric()
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_c46bd5b86bdf0e6783b59413306f05d9020e0adfd665346468876ec3fae54b62_d.txt',
 'azureml-logs/65_job_prep-tvmps_c46bd5b86bdf0e6783b59413306f05d9020e0adfd665346468876ec3fae54b62_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_c46bd5b86bdf0e6783b59413306f05d9020e0adfd665346468876ec3fae54b62_d.txt',
 'logs/azureml/93_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log']

In [14]:
best_run.get_metrics()

{'Regularization Strength:': 50.0,
 'Max iterations:': 300,
 'Accuracy': 0.9091047040971169}

In [15]:
best_run.get_details()

{'runId': 'HD_bc0db419-a96f-49dc-bd43-6acfe8e1d15b_1',
 'target': 'my-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-21T10:03:19.187599Z',
 'endTimeUtc': '2021-09-21T10:04:09.267944Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '5e065b8c-6ca0-423b-b1a7-c0e1a2a1e085',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '50', '--max_iter', '300'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'my-compute-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'en

In [16]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files([path])

In [17]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [18]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=ds,
    label_column_name="y",
    n_cross_validations=2)

In [19]:
# Submit your automl run

automl_run = exp.submit(automl_config)

RunDetails(automl_run).show()

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_1f431984-cacf-426d-950f-4de93907c791,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [20]:
best_run, fitted_model = automl_run.get_output()

In [21]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_1f431984-cacf-426d-950f-4de93907c791_30,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [22]:
best_run.get_details()

{'runId': 'AutoML_1f431984-cacf-426d-950f-4de93907c791_30',
 'status': 'Completed',
 'startTimeUtc': '2021-09-21T10:40:02.579187Z',
 'endTimeUtc': '2021-09-21T10:40:40.681572Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-project\',\'compute_target\':\'local\',\'subscription_id\':\'5a4ab2ba-6c51-4805-8155-58759ad589d8\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_1f431984-cacf-426d-950f-4de93907c791_30","experiment_name":null,"workspace_name":"quick-starts-ws-158927","subscription_id":"5a4ab2ba-6c51-4805-8155-58759ad589d8","resource_group_name":"aml-qui

In [23]:
best_run.get_metrics()

{'norm_macro_recall': 0.5148376071225602,
 'average_precision_score_micro': 0.9809584239077477,
 'average_precision_score_macro': 0.8217060249871734,
 'f1_score_macro': 0.7753628544074017,
 'AUC_weighted': 0.9454236818996533,
 'precision_score_micro': 0.9165705614567528,
 'f1_score_weighted': 0.9135512662745275,
 'recall_score_micro': 0.9165705614567528,
 'precision_score_macro': 0.7974608526639644,
 'log_loss': 0.24165258519756394,
 'recall_score_weighted': 0.9165705614567528,
 'AUC_macro': 0.9454236818996534,
 'f1_score_micro': 0.9165705614567526,
 'balanced_accuracy': 0.7574188035612801,
 'weighted_accuracy': 0.9561093967989045,
 'AUC_micro': 0.9801833393586181,
 'matthews_correlation': 0.5534069401018484,
 'precision_score_weighted': 0.9115967120928353,
 'accuracy': 0.9165705614567528,
 'average_precision_score_weighted': 0.9544757373518337,
 'recall_score_macro': 0.7574188035612801,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_1f431984-cacf-426d-950f-4de93907c7

In [24]:
best_run.get_file_names()

['accuracy_table',
 'confusion_matrix',
 'explanation/2945bec3/classes.interpret.json',
 'explanation/2945bec3/expected_values.interpret.json',
 'explanation/2945bec3/features.interpret.json',
 'explanation/2945bec3/global_names/0.interpret.json',
 'explanation/2945bec3/global_rank/0.interpret.json',
 'explanation/2945bec3/global_values/0.interpret.json',
 'explanation/2945bec3/local_importance_values.interpret.json',
 'explanation/2945bec3/local_importance_viz.interpret.json',
 'explanation/2945bec3/per_class_names/0.interpret.json',
 'explanation/2945bec3/per_class_rank/0.interpret.json',
 'explanation/2945bec3/per_class_values/0.interpret.json',
 'explanation/2945bec3/rich_metadata.interpret.json',
 'explanation/2945bec3/true_ys_viz.interpret.json',
 'explanation/2945bec3/visualization_dict.interpret.json',
 'explanation/2945bec3/ys_pred_proba_viz.interpret.json',
 'explanation/2945bec3/ys_pred_viz.interpret.json',
 'explanation/b0a70bef/classes.interpret.json',
 'explanation/b0a70b

In [27]:
# Cluster cleanup
compute_target.delete()